# How to roll your own scikit-learn compatible algorithm


In my presentation, I showed how you could simply create a class subclassed from **object**, provide the following interface, and have code which worked with most scikit-learn utility functions/classes

```
def __init__(self, paramA=defaultvalA, paramB=defaultvalB)
  
def fit(self, X, y, **option_fit_params)

def predict(self, X)

def score(self, X, y, sample_weight=None)

def get_params(self, deep=True)

def set_params(self, **params)
```


## What I've learned since my presentation

If I had instead subclassed from sklearn's  `base.BaseEstimator`, I would have inherited:

* `get_params()`  - BaseEstimator introspects your __init__() and returns a dict of your named params mapped to their values

* `set_params()`  - this method is always the same

* `__repr__()`, which makes use of `get_params()`


If I had made use of `base.ClassifierMixin`, I would have inherited:
* `score()` - which uses `sklearn.metrics.accuracy_score()`

It automatically sets the following attribute for you:
* `estimator_type_ = "classifier"`

If instead you're implementing a regression estimator, you could make use of `base.RegressorMixin` which provides
* `score()` which uses `sklearn.metrics.r2_score`

and sets the attribute:
* `estimator_type_ = "regressor"`

### Some serious reduction in boilerplate!
Making use of these classes and mixins allows you spend more time on implementing your actual algoritm, meaning the only public methods you need to implement are:

```
def __init__(self, paramA=defaultvalA, paramB=defaultvalB)
    """you shouldn't do any modification of the param values passed in as they can
    also be set by the set_params() method.  do sanity checking/modification of values
    in fit()
    """
    self.paramA = paramA
    self.paramB = paramB


def fit(self, X, y, **option_fit_params)
    """train your model
       do sanity checking of X, y, and parameters settable in __init__() here
       as those parameters can also be passed in via set_params()
    """


def predict(self, X)
    """to make use of your model (return a classication or some value from your regressor"""

```


In [1]:
# installs 'PghML' in sys.path (if it isn't there already)
import setup_sys_path 

# loader function for my dataset
from pgh_ml_py.datasets import load_banknote_authentication
# my decision tree implementation
from pgh_ml_py.sklearn_compat.tree.cart_decision_tree import CartDecisionTreeClassifier, display_tree

import numpy as np
import pandas as pd
# useful sklearn functions/Classes which we wish to be able to leverage (the point of making our code compatible)
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold, train_test_split
from sklearn.preprocessing import scale

In [2]:
dataset = load_banknote_authentication()

In [3]:
df = dataset.dataframe
X = dataset.data
y = dataset.target


In [4]:
train_X, test_X, train_y, test_y = train_test_split(X, y)

In [5]:
clf = CartDecisionTreeClassifier(criterion="entropy", max_depth=5, min_samples_split=5)

In [6]:
clf.fit(train_X, train_y)

CartDecisionTreeClassifier(criterion='entropy', max_depth=5,
              min_samples_split=5, splitter=u'best')

### Following sklearn's conventions for decision trees, my implementation's fit method sets the following 2 attributes:

* `clf.tree_`  - the underlying representation of the decision tree

* `clf.classes_` - the set of unique classes in y



### Here I call my ad_hoc function, display_tree(), which understands my decision trees representation makes use of these attributes

In [8]:
display_tree(clf.tree_, clf.classes_)

if feat[3] <= 2.450: (impurity: 0.606 num_samples: 1029 [570 459])
T-> 0
F-> 0


## Ok, great, but let's actually try and do something with my tree.  Let's call predict() on it

In [ ]:
clf.predict(test_X[0])

## yikes!

What's happening here ??? As you can see, it *works* but spits out an ugly deprecation warning

Sklearn classfiers predict methods expect an **array** of rows, so if we're passing in a single row of data we simply need to pass it as [row]


In [ ]:
clf.predict([test_X[0]])

### And let's see how accurate my tree is by passing in full test data set

In [7]:
clf.score(test_X, test_y)

0.55976676384839652

### Ok. 
That's fine for demonstrating how to fit/predict/score a single tree, but let's do a cross validation with 5 folds
#### New
I've discovered that cross_val_score() and some other cross-validation functions/Class allow you to use all cpus/cores by passing a named-parameter `n_jobs=-1` (otherwise it defaults to 1)

In [ ]:
cross_val_score(clf, dataset.data, dataset.target, n_jobs=-1, cv=5)

### Hmm. something looks **very** wrong here
The original code scored ~80%.  Did I break something in my refactoring?  Lets take a look at the data (like you really **should** do prior to doing anything)

In [ ]:
df.head()

In [ ]:
df.tail()

## Ok,  I *think* I'm seeing a pattern. Let's print out some more of the dataset to make sure I'm not hallucinating

In [ ]:
df

### Here's the problem with what I did

It turns out that my original function which created k-folds was randomizing (shuffling) the order of records, but that's not happening here

As you can see, **all** of the rows labeled **0** are in the **1st half** of the dataset while all the rows labeled **1** are in the **2nd half** of the dataset.  

By default, if you simply pass in an int for the cv param it uses KFold which doesn't deal with this. 

### Let's make use of StratifiedKFold instead to make sure that all of our folds have the classes balanced

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=np.random.RandomState(1))

In [ ]:
cross_val_score(clf, dataset.data, dataset.target, n_jobs=-1, cv=cv)

### Ok, great!  These values are pretty much in sync with the original blog post I based this off of.  That's a relief - I didn't break anything in all of my refactoring.

So far, I've simply made use of my class using it's default values of max_depth=5 and min_samples_split=20

### Let's make use of sklearn's GridSearchCV to try automatically optimize values for these parameters. 

Of course, this can take a while, so I'll keep the ranges of values to a reasonable size

In [ ]:
parameters = {'max_depth': range(3, 6), 'min_samples_split': range(2, 21, 2)}

In [ ]:
dt = CartDecisionTreeClassifier()

In [ ]:
clf = GridSearchCV(dt, parameters, cv=cv, n_jobs=-1, verbose=True)
print clf

In [ ]:
clf.fit(X, y)

### After running fit(), the GridSearchCV has a bunch of attributes set.  The ones I found most useful were:
* cv\_results\_      - lots of details which can be imported into pandas as a dataframe
* best\_score\_      - score of the best result
* best\_params\_     - dict of the best parameter values discovered
* best\_estimator\_  - the best estimator object (useful as you can inspect it)

In [ ]:
df2 = pd.DataFrame(clf.cv_results_)

In [ ]:
df2

In [ ]:
print """
best score: %f
best params: %s
""" % (clf.best_score_, clf.best_params_)

In [ ]:
best_tree = clf.best_estimator_

In [ ]:
display_tree(best_tree.tree_, best_tree.classes_)

## Going Forward

### More attributes.
Decision Trees in sklearn provide the following attributes.  What I'm not sure about is whether they are simply useful 
for allowing the model developer to inspect what was learned, or if certain utility functions/Classes actually make use of them.

* `classes_` - I implemented this one. simply the unique set of labels 


* `tree_` - I implemented this as well, simply provides access to the data structure which represents the tree


* `n_features_` -  I *believe* this should be the number of columns in X, or simply `X.shape[1]`


* `n_outputs_` - I *believe* this represents how many columns are in y, which is typically 1, but is computed as `y.ndim`  Perhaps this is more useful when doing multi-label/class/output stuff


* `n_classes_` - I *believe* this should simply be  `len(classes_)`


* `max_features_` - how many features were considered when determining the best splits

Also they provide more named-parameters in their constructors. I'm currently providing:
* `max_depth`


* `min_samples_split`

### More Hyper-Parameters
Scikit-learn decision trees provide the following named parameters in their constructors:

* `max_depth=None` - I implemented this one, defaults to unlimited depth

* `min_samples_split=2` - I implented this one, The minimum number of samples required for a branch to split


* For classifiers:
    * `criterion="gini"` - the other choice being "entropy". I could add support for using this as my metric for selecting the best split


* For regressors:
    * `criterion="mse"`  - the other choices being "friedman_mse", and "mae"
 
 
* `splitter="best"`  the other choice being "random". My implementation is "best".  I *believe* "random" would be useful for generating numerous trees for Random Forests


* `max_features` defaults to the number of features, different values determine how many features should be considered when trying to determine the best split


* `min_samples_leaf` the minimum number of samples required to represent a leaf node. I'm guessing this is used to prune the parent node and replace it with a leaf


* `max_leaf_nodes=None` - grows tree with 'max_leaf_nodes' in best-first fashion. best nodes are defined as relative reduction in impurity. If none, unlimited number of leaf nodes


* `min_weight_fraction_leaf=0` - minimum weighted fraction of the sum total of weights (of all input samples) required to be a leaf node. 0 means samples all have the same weight


* `class_weight=None`  if 'None' all classes have weight of 1, if "balanced" each class's weight is inversely proportional to class frequencies in 'y'. It can also be a dict of {class_label: weight, ...} or for multi-label a list of dicts in this format


* `random_state=None` - scikit-learn always randomizes the feature indices to use when calculating splits, even if max_features is the same as n_features_.  this parameter simply gives you control to make multiple runs deterministic


* `min_impurity_split=1e-7` threshold to prevent splits if a node's impurity is below, and instead generate a leaf node


* `presort=False` whether to presort the data to attempt to speed up finding the best splits. A setting of True may slow down fitting of large datasets (if max_depth is too high), but may speed up things up for small datasets 


Perhaps, by incorporating such hyper-parameters into my algorithm, and thus being able optimize for them,  I can improve the trees I generate.  

Again, my purpose isn't to re-implement scikit-learn's decision trees, but rather implement an algoritm which is compatable with sklearn, so that I can explore how the algorithm works.  Figuring out what additional hyper-parameters are relevant to decision trees would simply allow me to explore improving the algorithm.  Perhaps other hyper-parameters could be found from different implementations or my own ideas.


## References

Original blog post I got the algorithm from: http://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/

sklearn documentation on rolling your own estimator: http://scikit-learn.org/stable/developers/contributing.html#rolling-your-own-estimator


Gini index calculation
$$count_k = 1/ Nm \sum_{x_i in Rm} I(yi = k)$$
$$index = \sum_{k=0}^{K-1} count_k (1 - count_k)$$
$$      = 1 - \sum_{k=0}^{K-1} count_k ** 2$$

Entropy calculation
same calculation for $count_k$

cross entropy
$$crossEntropy = -\sum_{k=0}^{K-1} count_k log(count_k)$$

In [ ]:
arr = np.array([2,3,4,5])

In [ ]:
idx = np.where(arr < 3)


In [ ]:
idx


In [ ]:
arr[idx]


In [ ]:
help(np.bincount)


In [ ]:
arr


In [ ]:
arr = np.array(np.arange(7,10))


In [ ]:
np.arange(7,10)

In [ ]:
arr = np.array([1.91240001, 2.4333, 4.2442424, 4.35353535])


In [ ]:
arr

In [ ]:
np.where(np.isclose(arr, 1.9124001))

In [ ]:
arr = np.array([1,0,1,1,0,0,1,1,0,0,0,1,1])


In [ ]:
_, freqs = np.unique(arr, return_counts=True)

In [ ]:
freqs


In [ ]:
total = len(arr)

In [ ]:
total

In [ ]:
probs = freqs / total

In [ ]:
probs


In [ ]:
probs = freqs / float(total)

In [ ]:
probs
